In [1]:
import scanpy as sc
import stlearn as st
import numpy as np
import csv
import pickle
from scipy import sparse
import scipy.io as sio
import matplotlib
matplotlib.use('Agg')
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap, to_hex, rgb2hex
from typing import List
import qnorm
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from collections import defaultdict
import pandas as pd
import gzip
from kneed import KneeLocator
import copy 
#sys.path.append("/home/gw/code/utility/altairThemes/")
#if True:  # In order to bypass isort when saving
#    import altairThemes
#import altairThemes
import altair as alt
import argparse
import scipy.stats

spot_diameter = 89.43 #pixels

In [2]:
import sys
sys.path.append("/Users/victoriagao/Documents/MSc/Schwartz_lab/altair-themes/")
if True:  # In order to bypass isort when saving
    import altairThemes

In [3]:
def readCsv(x):
  """Parse file."""
  #colNames = ["method", "benchmark", "start", "end", "time", "memory"]
  df = pd.read_csv(x, sep=",")

  return df

def preprocessDf(df):
  """Transform ligand and receptor columns."""
  df["ligand-receptor"] = df["ligand"] + '-' + df["receptor"]
  df["component"] = df["component"] #.astype(str).str.zfill(2)

  return df


def plot(df):
  set1 = altairThemes.get_colour_scheme("Set1", len(df["component"].unique()))
  set1[0] = '#000000'
  base = alt.Chart(df).mark_bar().encode(
            x=alt.X("ligand-receptor:N", axis=alt.Axis(labelAngle=45), sort='-y'),
            y=alt.Y("count()"),
            color=alt.Color("component:N", scale = alt.Scale(range=set1)),
            order=alt.Order("component:N", sort="ascending"),
            tooltip=["component"]
        )
  p = base

  return p

### Below is all input #############################

In [4]:
data_name = 'PDAC_64630'

In [5]:
# read filtered matrix
adata_h5 = st.Read10X(path="/Users/victoriagao/local_docs/NEST/input", count_file='filtered_feature_bc_matrix.h5')
adata_h5

/Users/victoriagao/anaconda3/envs/NEST_try/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/victoriagao/anaconda3/envs/NEST_try/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 1406 × 36601
    obs: 'in_tissue', 'array_row', 'array_col', 'sum_counts', 'imagecol', 'imagerow'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [ ]:
# adata_h5.X

In [ ]:
# filter out the genes that are not expressed in any cell
sc.pp.filter_genes(adata_h5, min_cells=1)
# adata_h5

In [6]:
# get gene identifier - names
gene_ids = list(adata_h5.var_names)
gene_ids

['MIR1302-2HG',
 'FAM138A',
 'OR4F5',
 'AL627309.1',
 'AL627309.3',
 'AL627309.2',
 'AL627309.5',
 'AL627309.4',
 'AP006222.2',
 'AL732372.1',
 'OR4F29',
 'AC114498.1',
 'OR4F16',
 'AL669831.2',
 'LINC01409',
 'FAM87B',
 'LINC01128',
 'LINC00115',
 'FAM41C',
 'AL645608.6',
 'AL645608.2',
 'AL645608.4',
 'LINC02593',
 'SAMD11',
 'NOC2L',
 'KLHL17',
 'PLEKHN1',
 'PERM1',
 'AL645608.7',
 'HES4',
 'ISG15',
 'AL645608.1',
 'AGRN',
 'AL645608.5',
 'AL645608.8',
 'RNF223',
 'C1orf159',
 'AL390719.3',
 'LINC01342',
 'AL390719.2',
 'TTLL10-AS1',
 'TTLL10',
 'TNFRSF18',
 'TNFRSF4',
 'SDF4',
 'B3GALT6',
 'C1QTNF12',
 'AL162741.1',
 'UBE2J2',
 'LINC01786',
 'SCNN1D',
 'ACAP3',
 'PUSL1',
 'INTS11',
 'AL139287.1',
 'CPTP',
 'TAS1R3',
 'DVL1',
 'MXRA8',
 'AURKAIP1',
 'CCNL2',
 'MRPL20-AS1',
 'MRPL20',
 'AL391244.2',
 'ANKRD65',
 'AL391244.1',
 'TMEM88B',
 'LINC01770',
 'VWA1',
 'ATAD3C',
 'ATAD3B',
 'ATAD3A',
 'TMEM240',
 'SSU72',
 'AL645728.1',
 'FNDC10',
 'AL691432.4',
 'AL691432.2',
 'MIB2',
 'MMP

In [7]:
# access coordinates stored in adata_ha using the key spatial
coordinates = adata_h5.obsm['spatial']
coordinates

array([[ 9283, 12018],
       [11004, 10622],
       [ 8053,  6963],
       ...,
       [12040, 10617],
       [ 9801, 12115],
       [ 3230,  9762]])

In [8]:
# get cell identifier - barcode
cell_barcode = np.array(adata_h5.obs.index)
temp = adata_h5.X

#### Make Barcode_info

In [9]:
i=0
barcode_serial = dict()
for cell_code in cell_barcode:
    barcode_serial[cell_code] = i
    i = i + 1

i=0
barcode_info=[]
for cell_code in cell_barcode:
    barcode_info.append([cell_code, coordinates[i,0],coordinates[i,1], 0]) # last entry will hold the component number later
    i=i+1
    

In [10]:
barcode_info_dict = {item[0]: [item[1], item[2]] for item in barcode_info}

#### load Pathologist annotation

In [11]:
if data_name == 'PDAC_64630':
    pathologist_label_file='/Users/victoriagao/local_docs/NEST/IX_annotation_artifacts_PDAC64630.csv' #IX_annotation_artifacts.csv' #
    pathologist_label=[]
    with open(pathologist_label_file) as file:
        csv_file = csv.reader(file, delimiter=",")
        for line in csv_file:
            pathologist_label.append(line)	
    	
    barcode_type=dict() # record the type (annotation) of each spot (barcode)
    for i in range (1, len(pathologist_label)):
        barcode_type[pathologist_label[i][0]] = pathologist_label[i][1]
        

In [17]:
barcode_type

{'AAACCGGGTAGGTACC-1': 'tumor',
 'AAACCGTTCGTCCAGG-1': 'tumor',
 'AAACGAGACGGTTGAT-1': 'stroma_deserted',
 'AAACTGCTGGCTCCAA-1': 'stroma_deserted',
 'AAAGACCCAAGTCGCG-1': 'stroma_deserted',
 'AAAGGCTCTCGCGCCG-1': 'stroma_deserted',
 'AAAGGGATGTAGCAAG-1': 'tumor',
 'AAAGGGCAGCTTGAAT-1': 'stroma_deserted',
 'AAAGTAGCATTGCTCA-1': 'stroma_deserted',
 'AAAGTGTGATTTATCT-1': 'stroma_deserted',
 'AAATACCTATAAGCAT-1': 'stroma_deserted',
 'AAATAGGGTGCTATTG-1': 'tumor',
 'AAATCGTGTACCACAA-1': 'tumor',
 'AAATGGTCAATGTGCC-1': 'stroma_deserted',
 'AAATTAACGGGTAGCT-1': 'stroma_deserted',
 'AAATTGATAGTCCTTT-1': 'stroma_deserted',
 'AAATTTGCGGGTGTGG-1': 'stroma_deserted',
 'AACAACTGGTAGTTGC-1': 'stroma_deserted',
 'AACAATACATTGTCGA-1': 'Artifact',
 'AACAATTACTCTACGC-1': 'tumor',
 'AACACACGCTCGCCGC-1': 'stroma_deserted',
 'AACAGCTGTGTGGCAA-1': 'Artifact',
 'AACAGGATGGGCCGCG-1': 'stroma_deserted',
 'AACCAAGACTTCTCTG-1': 'tumor',
 'AACCCAGAGACGGAGA-1': 'stroma_deserted',
 'AACCGAGCTTGGTCAT-1': 'Artifact',

### Above is all input 

### Below is output

#### Read NEST output in csv format

In [12]:
filename_str = 'NEST_combined_output_PDAC_64630'+'.csv'
inputFile = '/Users/victoriagao/local_docs/NEST/output/'+filename_str
df = pd.read_csv(inputFile, sep=",")
csv_record_final = df.values.tolist()
df_column_names = list(df.columns)
csv_record_final = [df_column_names] + csv_record_final

component_list = dict()
for record_idx in range (1, len(csv_record_final)-1): #last entry is a dummy for histograms, so ignore it.
    record = csv_record_final[record_idx]
    i = record[6]
    j = record[7]
    component_label = record[5]
    barcode_info[i][3] = component_label 
    barcode_info[j][3] = component_label
    component_list[component_label] = ''

component_list[0] = ''
unique_component_count = len(component_list.keys())

In [66]:
len(csv_record_final)

1094

###  Histogram plotting 

In [55]:
#### i.e. plot only regions: tumor-stroma, stroma-tumor
filtered_cells = []

for record_idx in range (1, len(csv_record_final)-1): #last entry is a dummy for histograms, so ignore it.
    if ((barcode_type[csv_record_final[record_idx][0]] == 'tumor' and barcode_type[csv_record_final[record_idx][1]] != 'tumor') or (barcode_type[csv_record_final[record_idx][0]] != 'tumor' and barcode_type[csv_record_final[record_idx][1]] == 'tumor')):
        filtered_cells.append(csv_record_final[record_idx])

filtered_cells_dict = {tuple(row[:-2]): row for row in filtered_cells}

# Iterate through the rows in list2 and find matches in list1_dict
matched_rows = []
for row in csv_record_final:
    key = tuple(row[:-2])
    if key in filtered_cells_dict:
        matched_rows.append(filtered_cells_dict[key])
# matched_rows = [['from_cell','to_cell','ligand','receptor','attention_score','component','from_id','to_id']] + matched_rows
# print("Matched rows:", matched_rows)

In [56]:
matched_rows

[['GAATTTCTCGCTGCAG-1',
  'AACTTGCCCGTATGCA-1',
  'FN1',
  'RPSA',
  0.929611313343048,
  14,
  696,
  36],
 ['GCGGACCGCGTTGTGG-1',
  'AATAGAACAGAGTGGC-1',
  'THBS2',
  'LRP1',
  0.9100834131240844,
  23,
  808,
  63],
 ['AGTCTCACAAGACTAC-1',
  'AATGTGCCCGAGGTGT-1',
  'PTPRF',
  'RACK1',
  0.912053906917572,
  10,
  228,
  74],
 ['CCGTATCTCGTCGTAG-1',
  'AATGTGCCCGAGGTGT-1',
  'PTPRF',
  'RACK1',
  0.9118041157722472,
  10,
  474,
  74],
 ['TTATGAATGAAAGGGA-1',
  'ACATCGATCGTTTACC-1',
  'LGALS3',
  'ITGB4',
  0.9117532134056092,
  28,
  1332,
  103],
 ['CCGACGGGCATGAGGT-1',
  'AGCTGTAACCTCAATC-1',
  'FN1',
  'RPSA',
  0.9296771287918092,
  10,
  458,
  201],
 ['TACTTGTTAGTAGTCC-1',
  'AGTGAGACTTCCAGTA-1',
  'RPS19',
  'RPSA',
  0.9302722811698914,
  44,
  1072,
  231],
 ['TATTCGTGCCAGAATA-1',
  'ATATCTTAGGGCCTTC-1',
  'FN1',
  'RPSA',
  0.9292991638183594,
  48,
  1119,
  266],
 ['CAGTTCCGCGGGTCGA-1',
  'ATTCAGTAGCAGGGTC-1',
  'RPS19',
  'RPSA',
  0.9301169514656068,
  12,
  393,
  317

In [103]:
alt.themes.register("publishTheme", altairThemes.publishTheme)
alt.themes.enable("publishTheme")

## Defult whole sample 



columns = ['from_cell','to_cell','ligand','receptor','attention_score','component','from_id','to_id'] # make csv_record_final into a dataframe
df = pd.DataFrame(matched_rows, columns=columns)

# Set component of interest
target_component = 15
df_target = pd.DataFrame()
df_target = df[df['component'] == target_component]
# df.loc[df['component'] == 4] = df_4
df_target_processed = preprocessDf(df_target)

if target_component in df['component']:
    print("ok")

## Plot only component 13
# df_13 = pd.DataFrame()
# df_13 = df.loc[df['component'] == 13]
# df_13_processed = preprocessDf(df_13)

df_15 = pd.DataFrame()
df_15 = df[df['component'] == 15]
# df.loc[df['component'] == 4] = df_4
df_15_processed = preprocessDf(df_15)

## Plot only specified region of sample 
# x_boundary = (11000, 13000)
# y_boundary = (12000, 13000)

# filtered_cells = []
# plot_data = []

# # Filter each cell by their coordinates
# for cell_id, (x, y) in barcode_info_dict.items():
#     if x_boundary[0] <= x <= x_boundary[1]:
#         filtered_cells.append((cell_id, (x, y)))
        
# # For all filtered cells, match to csv_record_final - interaction pairs
# # if either the ligand or receptor is a cell within range, the interaction will be included
# for cell_id, _ in filtered_cells:
#     for row in csv_record_final:
#         if row[0] == cell_id or row[1] == cell_id: 
#             plot_data.append(row)

# for row in plot_data:
#     print(row)

# # assume csv_record_final is already filtered
# columns = ['from_cell','to_cell','ligand','receptor','attention_score','component','from_id','to_id'] # make csv_record_final into a dataframe
# df = pd.DataFrame(plot_data, columns=columns)
# df = preprocessDf(df)



ok


In [104]:
df_15

,from_cell,to_cell,ligand,receptor,attention_score,component,from_id,to_id,ligand-receptor


In [97]:
len(df_13_processed)

67

In [65]:
# if (df['component'] == 13).any():
#     print("ok")

In [93]:
# plot df
p = plot(df_4_processed)
outPath = '/Users/victoriagao/local_docs/NEST/output/new_hist.html'
p.save(outPath)